In [1]:
from cgi import test
from re import A
from model.line import LINE
import torch
import networkx as nx
import numpy as np
print(torch.cuda.is_available())

C:\Users\Omkar Sathe\AppData\Local\Temp\ipykernel_4708\3113790218.py:1: DeprecationWarning: 'cgi' is deprecated and slated for removal in Python 3.13
  from cgi import test


ModuleNotFoundError: No module named 'deepctr'

In [ ]:
def get_graph_0():
    G = nx.Graph()
    all_node = []
    all_edge = []
    with open("./data/gmd.edgelist") as f_edge:
        edge_content = f_edge.readlines()
        for edge in edge_content:
            node_1 = edge.strip("\n").split("\t")[0]
            node_2 = edge.strip("\n").split("\t")[1]
            weight = float(edge.strip("\n").split("\t")[2])
            if weight >0:
                all_edge.append((node_1, node_2))
            if node_1 not in all_node:
                all_node.append(node_1)
            if node_2 not in all_node:
                all_node.append(node_2)
    G.add_nodes_from(all_node)
    G.add_edges_from(all_edge)

    print("Nodes: ", G.number_of_nodes())
    print("Edges: ", G.number_of_edges())

    return G

In [ ]:
filea = open("data/X_train.txt")
x_train = filea.readlines()
fileb = open("data/y_train.txt")
y_train = fileb.readlines()

filea = open("data/X_test.txt")
X_test = filea.readlines()
fileb = open("data/y_test.txt")
y_test = fileb.readlines()

train = []
for i in range(len(x_train)):
        tup1 = (x_train[i].split()[0].split(",")[0],)
        tup2 = (x_train[i].split()[0].split(",")[1],)
        tup3 = (y_train[i].split()[0],)
        tup4 = tup1 + tup2 + tup3
        train.append(tup4)  # ('G721', 'DO144', '0')
           
test = []
for i in range(len(X_test)):
        tup1 = (X_test[i].split()[0].split(",")[0],)
        tup2 = (X_test[i].split()[0].split(",")[1],)
        tup3 = (y_test[i].split()[0],)
        tup4 = tup1 + tup2 + tup3
        test.append(tup4)  # ('G721', 'DO144', '0')

In [ ]:
G = get_graph_0()
# G = get_graph()
model = LINE(G, embedding_size=64, order='second')
model.train(batch_size=1024, epochs=1, verbose=2)
embeddings = model.get_embeddings()
embeddings_data = np.array(embeddings)

X_train_gd = []
for i in train:
    gd_feature = np.concatenate((embeddings[i[0]],embeddings[i[1]]))
    X_train_gd.append(gd_feature) 
X_train_gd = np.array(X_train_gd)
np.save('data/X_train_gmd_line.npy',X_train_gd)
print(X_train_gd.shape)

X_test_gd = []
for i in test:
    gd_feature = np.concatenate((embeddings[i[0]],embeddings[i[1]]))
    X_test_gd.append(gd_feature) 
X_test_gd = np.array(X_test_gd)
np.save('data/X_test_gmd_line.npy',X_test_gd)
print(X_test_gd.shape)

print("Success!")